In [207]:
#!pip install selenium
#!pip install webdriver-manager
#!pip install openpyxl

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from datetime import datetime, timedelta

In [15]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

In [16]:
stock_news_urls = [
    
    "https://www.investing.com/equities/advanced-semiconductor-engineering-news"
]



In [17]:
# 날짜 형식 정의
date_format = "%b %d, %Y"

# 기사 데이터 저장 리스트
articles = []


In [18]:
# 주식 뉴스 URL 리스트
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

for url in stock_news_urls:
    driver.get(url)
    time.sleep(5)  # 페이지 로드 대기

# 기사 데이터 저장 리스트
articles = []

In [19]:
def collect_articles(driver):
    for i in range(1, 11):  # 1부터 10까지 반복
        try:
            # 기사 영역 XPath
            article_xpath = f"//ul/li[{i}]/article"
            print(f"Trying to find article element with XPath: {article_xpath}")
            article_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, article_xpath))
            )

            # 헤드라인 XPath
            headline_xpath = f"{article_xpath}/div/a"
            print(f"Trying to find headline with XPath: {headline_xpath}")
            headline = driver.find_element(By.XPATH, headline_xpath).text

            # 게시 날짜 XPath
            date_xpath = f"{article_xpath}/div/ul/li[2]/time"
            print(f"Trying to find date with XPath: {date_xpath}")
            date_str = driver.find_element(By.XPATH, date_xpath).text

            # 날짜 형식 처리
            try:
                date_obj = datetime.strptime(date_str, date_format)
            except Exception:
                print(f"Unable to parse date: {date_str}. Using today's date instead.")
                date_obj = datetime.now()  # 날짜 형식이 맞지 않으면 오늘 날짜로 처리

            # 기사 데이터 저장
            articles.append({
                "headline": headline,
                "date": date_obj,
                "comments": []
            })

            # 댓글 클릭 XPath
            comment_icon_xpath = f"{article_xpath}/div/ul/li[3]/a"
            print(f"Trying to find comment icon with XPath: {comment_icon_xpath}")
            if driver.find_elements(By.XPATH, comment_icon_xpath):
                article_url = driver.current_url
                driver.find_element(By.XPATH, comment_icon_xpath).click()
                time.sleep(2)  # 페이지 로드 대기

                # 댓글 수집
                collect_comments(driver, i)

                # 원래 기사 페이지로 돌아가기
                driver.get(article_url)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, article_xpath))
                )
                time.sleep(2)  # 페이지 로드 대기

        except TimeoutException:
            print(f"TimeoutException: Unable to locate element for article {i} with XPath: {article_xpath}")
        except NoSuchElementException:
            print(f"NoSuchElementException: Unable to locate element for article {i} with XPath: {article_xpath}")
        except Exception as e:
            print(f"Error collecting article {i}: {e}")
            continue

In [20]:
def collect_comments(driver, article_index):
    try:
        # 댓글 아이콘 클릭 (이미 클릭한 상태라고 가정)
        time.sleep(2)  # 페이지 로드 대기

        # 'View all comments' 버튼 클릭
        show_all_comments_button_xpath = "/html/body/div[1]/div[2]/div[2]/div[2]/div[1]/div/div[13]/div/div/div[3]"
        if driver.find_elements(By.XPATH, show_all_comments_button_xpath):
            driver.find_element(By.XPATH, show_all_comments_button_xpath).click()
            time.sleep(2)  # 댓글 로드 대기

        # 댓글 수집
        comments = []
        i = 1
        while True:
            comment_xpath = f"/html/body/div[1]/div[2]/div[2]/div[2]/div[1]/div/div[13]/div/div/div[{i}]/div[1]/div[2]"
            if driver.find_elements(By.XPATH, comment_xpath):
                comment = driver.find_element(By.XPATH, comment_xpath).text
                comments.append(comment)
                i += 1
            else:
                break  # 댓글이 더 이상 없으면 루프 종료

        # 기사 데이터에 댓글 추가
        articles[article_index - 1]["comments"] = comments

    except Exception as e:
        print(f"Error collecting comments for article {article_index}: {e}")

# 기사 수집 반복문
# 엔비디아 30전까지 = df_NVDI1 / 30부터 79까지 df_NVDI2 / 80부터 끝까지 df_NVDI3
page = 1
while True:
    collect_articles(driver)
    # 다음 페이지 버튼 클릭
    next_button_xpath = "/html/body/div[1]/div[2]/div[2]/div[2]/div[1]/div[3]/a[2]"
    if driver.find_elements(By.XPATH, next_button_xpath):
        driver.find_element(By.XPATH, next_button_xpath).click()
        time.sleep(5)  # 페이지 로드 대기
        page += 1
        
    else:
        break
    
    if not articles:
        break

    # 수집한 마지막 기사의 날짜가 1년 이상 지난 경우 중지
    if (datetime.now() - articles[-1]["date"]).days > 365:
        break

# 드라이버 종료
driver.quit()

# 수집한 기사 데이터 출력
for article in articles:
    print(article)

Trying to find article element with XPath: //ul/li[1]/article
Trying to find headline with XPath: //ul/li[1]/article/div/a
Trying to find date with XPath: //ul/li[1]/article/div/ul/li[2]/time
Trying to find comment icon with XPath: //ul/li[1]/article/div/ul/li[3]/a
Trying to find article element with XPath: //ul/li[2]/article
Trying to find headline with XPath: //ul/li[2]/article/div/a
Trying to find date with XPath: //ul/li[2]/article/div/ul/li[2]/time
Trying to find comment icon with XPath: //ul/li[2]/article/div/ul/li[3]/a
Trying to find article element with XPath: //ul/li[3]/article
Trying to find headline with XPath: //ul/li[3]/article/div/a
Trying to find date with XPath: //ul/li[3]/article/div/ul/li[2]/time
Trying to find comment icon with XPath: //ul/li[3]/article/div/ul/li[3]/a
Trying to find article element with XPath: //ul/li[4]/article
Trying to find headline with XPath: //ul/li[4]/article/div/a
Trying to find date with XPath: //ul/li[4]/article/div/ul/li[2]/time
Trying to 

In [21]:
import pandas as pd

# 기사 데이터를 데이터프레임으로 변환
df = pd.DataFrame(articles, columns=["headline", "date", "comments"])

# 댓글을 문자열로 변환 (리스트를 문자열로 변환하여 저장)
df['comments'] = df['comments'].apply(lambda x: ' | '.join(x) if x else '')
df = df.sort_values(by='headline')

# 데이터프레임 저장 (CSV 파일)
df.to_csv('df_NVDA3.csv', index=False, encoding='utf-8-sig')

# 데이터프레임 출력
print(df)


                                            headline  \
1  ASE Industrial ADR earnings beat by $1.20, rev...   
4  ASE Industrial ADR earnings beat by $2.01, rev...   
6    ASE Technology reports dip in December revenues   
5  ASE Technology reports dip in Q4 net income, r...   
9  Advanced Semiconductor Engineering receives In...   
2  Aussie bourse operator ASX falls on earnings m...   
0  Earnings call: ASE Technology reports mixed Q1...   
3  Earnings call: ASE Technology reports mixed re...   
7  Technology one receives Investment Bank Analys...   
8  These stocks will benefit from Nvidia's new AI...   

                        date comments  
1 2024-04-25 00:00:00.000000           
4 2024-02-01 00:00:00.000000           
6 2024-01-09 00:00:00.000000           
5 2024-02-01 00:00:00.000000           
9 2024-04-26 00:00:00.000000           
2 2024-02-15 00:00:00.000000           
0 2024-04-29 00:00:00.000000           
3 2024-02-01 00:00:00.000000           
7 2024-05-24 14:13:22.9

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 1 to 8
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   headline  10 non-null     object        
 1   date      10 non-null     datetime64[ns]
 2   comments  10 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 320.0+ bytes


In [23]:
df['comments']

1    
4    
6    
5    
9    
2    
0    
3    
7    
8    
Name: comments, dtype: object